In [ ]:
def parse_html_payload(payload):
    """
    This function uses Beautiful Soup to read HTML data
    and return the text.  If the payload is plain text, then
    Beautiful Soup will return the original content
    """
    soup = BeautifulSoup(payload, 'html.parser')
    return str(soup.get_text()).encode('utf-8').decode('utf-8')

def parse_email(original_msg):
    result = {}
    #msg = parse_html_payload(original_msg)
    msg = Parser(policy=default).parsestr(original_msg)
    
    msg = parse_html_payload(original_msg)
    print(msg)
    
    body = msg.get_payload()
    tuple_result = tuple([str(result.get(column, None)) for column in columns])
    return body

In [ ]:
## This creates a user-defined function which can be used in Spark
parse_email_func = udf(lambda z: parse_email(z), email_struct)

def parse_emails(input_df):
    new_df = input_df.select(
        'username', 'id', 'original_msg', parse_email_func('original_msg').alias('parsed_email')
    )
    for column in columns:
        new_df = new_df.withColumn(column, new_df.parsed_email[column])
    
    new_df = new_df.drop('parsed_email')
    return new_df
        
class ParseEmailsTransformer(Transformer):
    def _transform(self, dataset):
        """
        Transforms the input dataset.

        :param dataset: input dataset, which is an instance of :py:class:`pyspark.sql.DataFrame`
        :returns: transformed dataset
        """
        return dataset.transform(parse_emails)

## Use the custom ParseEmailsTransformer, Tokenizer, and CountVectorizer 
## to create a spark pipeline 

parse = ParseEmailsTransformer()
tokenizer = Tokenizer(inputCol='original_msg', outputCol='words')
vectorizer = CountVectorizer(inputCol=tokenizer.getOutputCol(), outputCol='features')
#stages = parse
stages = [parse, tokenizer, vectorizer]

email_pipeline = Pipeline(stages=stages)

model = email_pipeline.fit(df)
result = model.transform(df)